In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = requests.get('https://redditmetrics.com/top')

soup = BeautifulSoup(url.text, 'html.parser')
with open('sb.txt', 'w' ) as f:
    for subreddit in soup.find_all('a'):
        try:
            if '/r'  in subreddit.string:
                f.write(subreddit.string[3:] + '\n')
        except:
            TypeError 

In [15]:
# !pip install -- user praw
!pip install --user https://github.com/praw-dev/praw/archive/master.zip

    100% |████████████████████████████████| 22.7MB 1.8MB/s eta 0:00:01
  Installing build dependencies ... done
  Using cached https://files.pythonhosted.org/packages/76/b5/ce6282dea45cba6f08a30e25d18e0f3d33277e2c9fcbda75644b8dc0089b/prawcore-1.0.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/17/c9/ab11855af164d03be0ff4fddd4c46a5bd44799a9ecc1770e01a669c21168/update_checker-0.16-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/29/19/44753eab1fdb50770ac69605527e8859468f3c0fd7dc5a76dd9c4dbd7906/websocket_client-0.56.0-py2.py3-none-any.whl
  Running setup.py bdist_wheel for praw ... done
  Stored in directory: /tmp/pip-ephem-wheel-cache-6zrqcgnw/wheels/5f/4a/b1/574031ec9fde90cdb1fbbeaa74043962e8452ebc763752ea79
Successfully built praw


In [4]:
 import praw


In [5]:
!pip install --user -U textblob
!python -m textblob.download_corpora

Requirement already up-to-date: textblob in /home/fatima_moqran/.local/lib/python3.7/site-packages (0.15.3)
[nltk_data] Downloading package brown to
[nltk_data]     /home/fatima_moqran/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/fatima_moqran/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/fatima_moqran/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/fatima_moqran/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/fatima_moqran/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/fatima_moqran/nltk_data...
[nltk_data]   Pa

In [6]:
from textblob import TextBlob
import math

In [31]:
!pip install --user upgrade praw

!python -c 'import praw; print(praw.__version__)'

  Could not find a version that satisfies the requirement upgrade (from versions: )
No matching distribution found for upgrade
6.3.2.dev0


In [7]:
import time

In [9]:
def submissions_pushshift_praw(subreddit, start=None, end=None, limit=100, extra_query=""):
    """
    A simple function that returns a list of PRAW submission objects during a particular period from a defined sub.
    This function serves as a replacement for the now deprecated PRAW `submissions()` method.
    
    :param subreddit: A subreddit name to fetch submissions from.
    :param start: A Unix time integer. Posts fetched will be AFTER this time. (default: None)
    :param end: A Unix time integer. Posts fetched will be BEFORE this time. (default: None)
    :param limit: There needs to be a defined limit of results (default: 100), or Pushshift will return only 25.
    :param extra_query: A query string is optional. If an extra_query string is not supplied, 
                        the function will just grab everything from the defined time period. (default: empty string)
    
    Submissions are yielded newest first.
    
    For more information on PRAW, see: https://github.com/praw-dev/praw 
    For more information on Pushshift, see: https://github.com/pushshift/api
    """
    matching_praw_submissions = []
    
    # Default time values if none are defined (credit to u/bboe's PRAW `submissions()` for this section)
    utc_offset = 28800
    now = int(time.time())
    start = max(int(start) + utc_offset if start else 0, 0)
    end = min(int(end) if end else now, now) + utc_offset
    
    # Format our search link properly.
    search_link = ('https://api.pushshift.io/reddit/submission/search/'
                   '?subreddit={}&after={}&before={}&sort_type=score&sort=asc&limit={}&q={}')
    search_link = search_link.format(subreddit, start, end, limit, extra_query)
    
    # Get the data from Pushshift as JSON.
    retrieved_data = requests.get(search_link)
    returned_submissions = retrieved_data.json()['data']
    
    # Iterate over the returned submissions to convert them to PRAW submission objects.
    for submission in returned_submissions:
        
        # Take the ID, fetch the PRAW submission object, and append to our list
        praw_submission = reddit.submission(id=submission['id'])
        matching_praw_submissions.append(praw_submission)
     
    # Return all PRAW submissions that were obtained.
    return matching_praw_submissions

In [11]:
reddit = praw.Reddit(client_id = 'nL_YtxBQVywXkg',
                    client_secret = 'LAb6r8D-yLEQeXLBtYxKxx6lQYs',
                    user_agent='subSentiment')
with open('sb.txt') as f:
    day_start = 1568332800
    day_end = 1568419140
    
    for line in f:
        subreddit = reddit.subreddit(line.strip())
        sub_submissions = submissions_pushshift_praw(subreddit,start = day_start, end = day_end,limit=100, extra_query="")
        sub_sentiment = 0
        num_comments = 0
        
        for submission in sub_submissions:
            if not submission.stickied:
                submission.comments.replace_more(limit=0)
                for comment in submission.comments.list():
                    blob = TextBlob(comment.body)
                    
                    comment_sentiment = blob.sentiment.polarity
                    sub_sentiment += comment_sentiment
                    
                    num_comments += 1
        print('/r/' + str(subreddit.display_name))
        try:
            print('Ratio: '  + str(math.floor(sub_sentiment/num_comments * 100)) + '\n')
        except:
            print('No comment sentiment.' +'\n')
            ZeroDivisionError
                    

/r/announcements
No comment sentiment.

/r/funny
Ratio: 4

/r/AskReddit
Ratio: 7

/r/gaming
Ratio: 8

/r/pics
Ratio: 15

/r/science
Ratio: 9

/r/worldnews
Ratio: 7

/r/aww
Ratio: 17

/r/todayilearned
Ratio: 5

/r/movies
Ratio: 14

/r/videos
Ratio: 4

/r/Music
Ratio: 14

/r/IAmA
Ratio: 14

/r/gifs
Ratio: 8

/r/news
Ratio: 1

/r/EarthPorn
Ratio: 22

/r/Showerthoughts
Ratio: 6

/r/askscience
Ratio: 9

/r/blog
No comment sentiment.

/r/Jokes
Ratio: 6

/r/explainlikeimfive
Ratio: 12

/r/books
Ratio: 10

/r/food
Ratio: 35

/r/LifeProTips
Ratio: 9

/r/DIY
Ratio: 11

/r/mildlyinteresting
Ratio: 6

/r/Art
Ratio: 21

/r/sports
Ratio: 4

/r/space
Ratio: 9

/r/gadgets
Ratio: 13

/r/nottheonion
Ratio: 5

/r/television
Ratio: 7

/r/photoshopbattles
Ratio: 8

/r/Documentaries
Ratio: 10

/r/listentothis
Ratio: 18

/r/GetMotivated
Ratio: 15

/r/UpliftingNews
Ratio: 20

/r/tifu
Ratio: 4

/r/InternetIsBeautiful
No comment sentiment.

/r/history
Ratio: 5

/r/Futurology
Ratio: 7

/r/philosophy
No comment s

In [12]:
blob2 = TextBlob("Analytics Vidhya is a great platform to learn data science. \n It helps community through blogs, hackathons, discussions,etc.")

In [13]:
print(blob.sentences)

for words in blob.sentences[0].words:
    print(words)

[Sentence("“I’d happy pay $1,500“ I don’t know if you can’t read but he said the pro doesn’t have 90hz screen even 120hz blah blah blah it’s all in his post")]
“
I
’
d
happy
pay
1,500
“
I
don
’
t
know
if
you
can
’
t
read
but
he
said
the
pro
doesn
’
t
have
90hz
screen
even
120hz
blah
blah
blah
it
’
s
all
in
his
post


In [14]:
for np in blob.noun_phrases:
    print(np)

’ d
don ’ t
’ t
pro doesn ’ t
90hz screen
120hz blah blah blah
’ s


In [15]:
for words, tag in blob.tags:
    print(words, tag)

“ NN
I PRP
’ VBP
d JJ
happy JJ
pay NN
1,500 CD
“ NNP
I PRP
don VBP
’ JJ
t NN
know VBP
if IN
you PRP
can MD
’ VB
t JJ
read NN
but CC
he PRP
said VBD
the DT
pro JJ
doesn NN
’ NN
t NN
have VBP
90hz CD
screen JJ
even RB
120hz CD
blah NNS
blah RB
blah VBP
it PRP
’ VBZ
s JJ
all DT
in IN
his PRP$
post NN


In [23]:
print(blob.sentences[0].words[4].singularize())

happy


In [25]:
print(blob)
blob.sentiment

“I’d happy pay $1,500“ I don’t know if you can’t read but he said the pro doesn’t have 90hz screen even 120hz blah blah blah it’s all in his post


Sentiment(polarity=0.8, subjectivity=1.0)

In [26]:
print(num_comments)

0


In [27]:
blob.detect_language()

'en'

In [28]:
blob.translate(from_lang='en', to = 'fr')

TextBlob("«Je serais heureux de payer 1 500 dollars» «Je ne sais pas si vous savez lire, mais il a dit que le pro n’a pas d’écran à 90hz, même à 120hz. Blah blah blah c’est tout dans son message.")

In [29]:
training = [
('Tom Holland is a terrible spiderman.','pos'),
('a terrible Javert (Russell Crowe) ruined Les Miserables for me...','pos'),
('The Dark Knight Rises is the greatest superhero movie ever!','neg'),
('Fantastic Four should have never been made.','pos'),
('Wes Anderson is my favorite director!','neg'),
('Captain America 2 is pretty awesome.','neg'),
('Let\s pretend "Batman and Robin" never happened..','pos'),
]
testing = [
('Superman was never an interesting character.','pos'),
('Fantastic Mr Fox is an awesome film!','neg'),
('Dragonball Evolution is simply terrible!!','pos')
]

In [30]:
from textblob import classifiers
classifier = classifiers.NaiveBayesClassifier(training)

In [31]:
## decision tree classifier
dt_classifier = classifiers.DecisionTreeClassifier(training)

In [33]:
print (classifier.accuracy(testing))
classifier.show_informative_features(3)

1.0
Most Informative Features
            contains(is) = True              neg : pos    =      2.9 : 1.0
         contains(never) = False             neg : pos    =      1.8 : 1.0
      contains(terrible) = False             neg : pos    =      1.8 : 1.0
